In [1]:
import pandas as pd
import numpy as np
import time
import mmcv
import json
import time
import os
import sys
import os.path as pt
from ensemble_boxes import weighted_boxes_fusion
import warnings
from yolo_inference import *
warnings.filterwarnings('ignore')

OVERLAP :  0.2


In [2]:
IMAGE_ROOT = '../../../../user_data/tmp_data/round-2/video_train/'
CKPT_PATH = './runs/train/fbody9/weights/best.pt'
OUT_ROOT = '../../../../user_data/tmp_data/tracking/MOT-PANDA/'

In [3]:
model = init_yolo_detector(CKPT_PATH,('full_body',), device='cuda:1')

Fusing layers... 


In [35]:
img_paths = {0:{},1:{},2:{},3:{}}
vid_nms = [x for x in os.listdir(IMAGE_ROOT) if not x.endswith('zip')]
for vid_nm in vid_nms:
    gpu_id = int(vid_nm.split('_')[0]) % 4
    img_root = os.path.join(IMAGE_ROOT, vid_nm)
    img_nms = os.listdir(img_root)
    img_paths[gpu_id][vid_nm] = sorted([os.path.join(img_root, img_nm) for img_nm in img_nms])

In [47]:
for vid_nm, paths in img_paths[0].items():
    det_lst = []
    for path in paths:
        frame = int(path.split("/")[-1].split("_")[-1].split(".")[0])
        yolov5_dets = simple_infer(path, model, device='cuda:1')
        for det in yolov5_dets:
            x1, y1, x2, y2, score = det
            det_lst.append([frame, -1, x1, y1, x2-x1, y2-y1, score])                        
        break
    break

In [65]:
det_out_path = os.path.join(out_root, vid_nm)+ '/det/yolov5_det.txt'

In [51]:
det_df = pd.DataFrame(det_lst)

In [66]:
det_df.to_csv(det_out_path, index=False, header=False)

In [67]:
det_out_path

'/raid/ly/Dataset/tianchi2021/user_data/tmp_data/tracking/MOT-PANDA/08_Xili_Street_1/det/yolov5_det.txt'